# 使用RNN文本分类

这部分将在IMDB评论集上训练一个RNN情感分类器。

## Setup

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

tf.__version__

导入 `matplotlib`，定义一个画图函数:

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

## 设置输入管道


IMDB电影评论数据集是一个二元分类数据集，所有评论要么是正面的，要么是负面的。

使用[TFDS]下载数据集.


In [ ]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_examples, test_examples = dataset['train'], dataset['test']

 数据集的 `info`包含编码器 (`tfds.features.text.SubwordTextEncoder`).

In [ ]:
encoder = info.features['text'].encoder

In [ ]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

文本编码器对字符串进行编码和解码，如果必要将使用字节编码。

In [ ]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

In [ ]:
assert original_string == sample_string

In [ ]:
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

## 准备训练数据

接下来创建编码字符串的批。使用 `padded_batch`方法0填充序列到批中最长长度:

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = (train_examples
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE, padded_shapes=([None],[])))

test_dataset = (test_examples
                .padded_batch(BATCH_SIZE,  padded_shapes=([None],[])))

next(iter(train_dataset))

注意：在**TensorFlow 2.2** 参数padded_shapes不再需要，缺省行为就是将所有轴填充到批中最大长度。

In [ ]:
train_dataset = (train_examples
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE))

test_dataset = (test_examples
                .padded_batch(BATCH_SIZE))

## 创建模型

创建一个`tf.keras.Sequential`模型，嵌入层作为模型的第一层。嵌入层对每个词保存一个向量。当调用时，它将词索引序列转换成向量序列。这些嵌入向量是可学习的。训练后，具有相似意义的词将有相似的向量。

这个索引查找比通过`tf.keras.layers.Dense`层传递单热向量要有效得多。

循环神经网络RNN迭代的处理序列中的元素。RNN将一个时间步的输出作为下一时间步的输入，如此循环往复.

`tf.keras.layers.Bidirectional`包装器和RNN一起使用，它使得RNN正向和反向处理序列，并将输出拼接concat起来，这可以帮助RNN学习长距离依赖。

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

编译模型以配置训练过程:

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

## 训练模型

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

上面模型没有对序列的填充进行掩蔽。如果在填充序列上训练，在未填充序列上测试，这可能导致偏差。理想情况下，你可以使用掩蔽来避免这些，但是，这对结果的影响较小。

如果预测>= 0.5, 评论是正面的，否则是负面的。

In [ ]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [ ]:
def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [ ]:
# 在未填充文本上预测.

sample_pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print(predictions)

In [ ]:
# 在填充文本上预测

sample_pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

## 堆叠多个LSTM层

Keras循环层通过`return_sequences`构造参数可以有两种不同的模式:

* 返回每个时间步连续输出完整序列（一个3D张量，形状为`(batch_size, timesteps, output_features)`）.
* 只返回输入序列最后的输出（一个2D张量，形状为`(batch_size, output_features)`）.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# 在没有填充的文本上预测.

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print(predictions)

In [ ]:
# 在填充文本上预测

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

以上例子可以使用其他的循环层，例如[GRU layers]， 你当然也可以对RNN进行定制，定制参考指南部分。